In [2]:
!pip install transformers datasets pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00


In [3]:
from transformers import pipeline
import sys
from datasets import load_dataset, Dataset, DatasetDict


In [4]:
model_name = "davidgaofc/SFT_Med_t"
generator = pipeline('text2text-generation', model=model_name, device=0)

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
dataset_name = "davidgaofc/RM_prompts"
dataset = load_dataset(dataset_name)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1640 [00:00<?, ? examples/s]

In [14]:
def get_responses(prompt, batch_size=8, max_length=200):
    # Generate two responses for the given prompt
    responses_1 = []
    responses_2 = []
    counter = 1
    for i in range(0, len(prompt), batch_size):
        print(counter)
        batch_prompts = prompt[i:i + batch_size]

        # Generating the first set of responses
        batch_responses_1 = generator(batch_prompts, max_length=max_length)
        responses_1.extend([response['generated_text'].strip() for response in batch_responses_1])

        # Generating the second set of responses
        batch_responses_2 = generator(batch_prompts, max_length=max_length)
        responses_2.extend([response['generated_text'].strip() for response in batch_responses_2])
        counter += 1

    return list(zip(responses_1, responses_2))

In [15]:
prompts = [entry["Question"] for entry in dataset["train"]]  # Replace 'text_field'
labels = [entry["label"] for entry in dataset["train"]]
    # Get responses in batches
batch_responses = get_responses(prompts)

records = []

for prompt, label, (response1,response2) in zip(prompts, labels, batch_responses):
  formatted_record = f"human: {prompt}\nassistant:\n1. {response1}\n2. {response2}"
  records.append({"Text": formatted_record, "Label": label})
# Convert the records to a DataFrame and save as CSV


1


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205


NameError: ignored

In [16]:
import pandas as pd
df = pd.DataFrame(records)
df.to_csv("model_responses.csv", index=False)

In [18]:
from huggingface_hub import notebook_login
notebook_login()

In [19]:
new_dataset = Dataset.from_csv("model_responses.csv")
new_dataset.push_to_hub("davidgaofc/RM_set_inout")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]